In [1]:
import os 
import sys
import json
sys.path.append(os.path.abspath(".."))
from velopix_wrappers.optimizers import BaseOptimizer
from velopix_wrappers.velopix_pipeline import TrackFollowingPipeline, GraphDFSPipeline, SearchByTripletTriePipeline
from typing import Any, Dict
import numpy as np

from typing import Any, Dict, Literal, List
import random
import numpy as np
from velopix_wrappers.optimizers import BaseOptimizer, pMap
from copy import deepcopy

In [2]:
class Node(object):
    def __init__(self, bounds, parent: Any = None):
        self.sum_reward = 0
        self.visited = 0
        self.children = []
        self.parent = parent
        self.bounds = bounds

    def add_child(self, child: Any):
        self.children.append(child)

In [ ]:
#return pmap based on bounds (split in middle) (rollout phase)
def returnPmap (bounds: Dict[str, Any], cfg: Dict[str, Any]) -> Dict[str, Any]:
    new_pmap = {}
    
    for key, (typ, _) in cfg.items():
        if typ is float:
            new_pmap[key] = (bounds[key][0] + bounds[key][1]) / 2
        elif typ is int:
            new_pmap[key] = (int)((bounds[key][0] + bounds[key][1]) / 2)
        elif typ is bool:
            new_pmap[key] = bounds[key]
        
    return new_pmap


def returnBounds (bounds: Dict[str, Any], cfg: Dict[str, Any]) -> List [Dict[str, Any]]: #TODO we currently split each of the bounds in half but we only want to split the axis with the 
                                                                                        # currently largest diameter, also its late my brain is cooked but should we skip over the keys that 
                                                                                        # are boolean as we currently try to split these? we also never noremalize anywhere.

    map = returnPmap(bounds, cfg)
    new_bounds = [deepcopy(bounds), deepcopy(bounds)]

    
    for key, (typ, _) in cfg.items():
        new_bounds[0][key][1] = map[key]
        new_bounds[1][key][0] = map[key]
        
        
    return new_bounds

In [ ]:
class PolyHoot(BaseOptimizer):
    def __init__(
        self,
        max_iterations: int = 100,
        objective: Literal["min", "max"] = "min"
    ):
        super().__init__(objective=objective, auto_eval={"autoEval": True, "nested": True, "weights": [1.0, 1.0, 1.0, -10.0]})
        self.max_iterations = max_iterations
        self.current_iteration = 0

        self.alfa = 5
        self.epsilon = 20
        self.eta = 0.5


        self.cfg = self._algorithm.get_config()
        self.bounds = self._algorithm.get_bounds()

        self.root = Node(bounds=self.bounds)  # Root node with no bounds
        self.nodes = [self.root]
        self.current_node = self.root

        self.param_num = 0

        for key, (typ, _) in self.cfg.items():
            if typ is not bool:
                self.param_num += 1

        self.nu = 4 * self.param_num
        self.ro = 1 / (4 * self.param_num)


    def init(self) -> pMap:
        """
        Initializes with a random point within bounds.
        """
        self.current_iteration += 1

        #pregenerate trees of bounds
        for key, (typ, _) in self.cfg.items():
            if typ is bool:
                current_leaves = [node for node in self.nodes if len(node.children) == 0]
                for node in current_leaves: # I think this is an infite loop no since we keep adding nodes to the nodes list in the loop we keep adding children forever no?
                    #also changed logic since we were overriding the original bounds.
                    bounds_false = deepcopy(node.bounds)
                    bounds_false[key] = 0 #TODO should this be an int or a true bool to ask group..
                    node1 = Node(bounds=bounds_false, parent=node)
                    
                    bounds_true = deepcopy(node.bounds)
                    bounds_true[key] = 1 #TODO should this be an int or a true bool to ask group..
                    node2 = Node(bounds=bounds_true, parent=node)
                    
                    #I think we also want to add these as children to the parent node yeah? or in this case the current node
                    node.children.append(node1)
                    node.children.append(node2)
                    
                    #leave unchanged..
                    self.nodes.append(node1)
                    self.nodes.append(node2)


        node = self.root
        depth = 0

        while len(node.children):
            node.visited += 1
            node = node.children[0]
            depth += 1


        node.visited += 1

        new_bounds = returnBounds(node.bounds, self.cfg)

        node.add_child(Node(bounds=new_bounds[0], parent=node))
        node.add_child(Node(bounds=new_bounds[1], parent=node))


        pmap = returnPmap(node.bounds, self.cfg)

        return pmap
    


    def next(self) -> pMap:
        """
        Evaluates the current configuration and returns a new one.
        """
        self.current_iteration += 1

        # Evaluate the current configuration (from previous init/next call)
        #score = self.objective_func([1.0, 1.0, 1.0, -10.0])
        #so we already increased the count of the nodes but we still want to backprop the scors which should happen after a run so at the start of 
        # next we get the last score in history and trace back up the stack accordingly or just use the score above but faster to get the score from history??? TODO ask team
        #TODO:backprop here FIXED?!?!
        if hasattr(self, 'score_history') and len(self.score_history) > 0:
            score = self.score_history[-1]
            current = self.current_node
            while current is not None:
                current.sum_reward += score
                current = current.parent
        
        node = self.root
        depth = 0

        while len(node.children):
            node.visited += 1
            if node.children[0].visited == 0:
                node = node.children[0]
            elif node.children[1].visited == 0:
                node = node.children[1]
            else:
                node1_score = (node.children[0].sum_reward / node.children[0].visited) + (self.current_iteration ** (self.alfa/self.epsilon)) * (node.children[0].visited ** (self.eta - 1)) + (self.nu * (self.ro ** depth))
                node2_score = (node.children[1].sum_reward / node.children[1].visited) + (self.current_iteration ** (self.alfa/self.epsilon)) * (node.children[1].visited ** (self.eta - 1)) + (self.nu * (self.ro ** depth))

                if node1_score > node2_score:
                    node = node.children[0]
                else:
                    node = node.children[1]

            depth += 1

        node.visited += 1

        new_bounds = returnBounds(node.bounds, self.cfg)

        node.add_child(Node(bounds=new_bounds[0], parent=node))
        node.add_child(Node(bounds=new_bounds[1], parent=node))


        pmap = returnPmap(node.bounds, self.cfg)
        #keep track of the leaf node we expanded and rolled out for the backprop we we do next again.
        self.current_node = node

            
        return pmap
    

    def is_finished(self) -> bool:
        """Determines if optimization is complete."""
        #TODO: possibly add a check for reaching target score
        #TODO check with team but I think here we also want to perform backprop since if were finished we wont performn the last next we need to backprop so we do it here instead
        finished = self.current_iteration >= self.max_iterations
        if finished:
            if hasattr(self, 'score_history') and len(self.score_history) > 0:
                score = self.score_history[-1]
                current = self.current_node
                while current is not None:
                    current.sum_reward += score
                    current = current.parent
                
        return finished
            

In [ ]:
events = []
n_files = 150

for i in range(0, n_files):
    if i == 51:
        """
        There's an issue with event 51 -> module_prefix_sum contains value 79 twice resulting in and indexing error when loading the event
        """
        print(f"Skipping problematic file: velo_event_{i}.json")
    else:    
        print(f"Loading file: velo_event_{i}.json")
        event_file = open(os.path.join("../DB/raw", f"velo_event_{i}.json"))
        json_data = json.loads(event_file.read())
        events.append(json_data) # type: ignore
        event_file.close()

In [ ]:
pipeline = TrackFollowingPipeline(events=events, intra_node=True) # type: ignore 

In [ ]:
Optimiser = PolyHoot(max_iterations=243, objective="min") # type: ignore
optimal_parameters = pipeline.optimise_parameters(Optimiser, max_runs=2430) # DO NOT remove max_runs, chances are that this will run forever

In [ ]:
print(optimal_parameters) # Note these are just here for example...